In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [3]:
# Generate summary statistics
df_market_data.describe()

In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)


---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
df_market_data_scaled = StandardScaler().fit_transform(df_market_data[[
        "price_change_percentage_24h", "price_change_percentage_7d",
       "price_change_percentage_14d", "price_change_percentage_30d",
       "price_change_percentage_60d", "price_change_percentage_200d",
       "price_change_percentage_1y"]])

In [6]:
#Column Names
df_market_data.columns

In [7]:
# Create a DataFrame with the scaled data
transformed_market_df = pd.DataFrame(df_market_data_scaled, 
                                     columns=["price_change_percentage_24h", "price_change_percentage_7d",
                                               "price_change_percentage_14d", "price_change_percentage_30d",
                                               "price_change_percentage_60d", "price_change_percentage_200d",
                                               "price_change_percentage_1y"])

# Copy the crypto names from the original data
transformed_market_df['Crypto_names'] = df_market_data.index

# Set the coinid column as index
transformed_market_df = transformed_market_df.set_index('Crypto_names')

# Display sample data
transformed_market_df.head()

---

### Find the Best Value for k Using the Original Data.

In [8]:
# Create a list with the number of k-values from 1 to 11
k=list(range(1,11))

In [9]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(transformed_market_df)
    inertia.append(k_model.inertia_)

In [10]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k_original": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_curve_df = pd.DataFrame(elbow_data)

In [11]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_curve_plot = elbow_curve_df.hvplot.line(
    x="k_original", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

elbow_curve_plot

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [12]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=0)

In [13]:
# Fit the K-Means model using the scaled data
model.fit(transformed_market_df)

In [14]:
# Predict the clusters to group the cryptocurrencies using the scaled data
cluster_predictions = model.predict(transformed_market_df)

# Print the resulting array of cluster values.
cluster_predictions 

In [15]:
# Create a copy of the DataFrame
transformed_market_df_copy = transformed_market_df.copy()

In [16]:
# Add a new column to the DataFrame with the predicted clusters
transformed_market_df_copy['Prediction'] = cluster_predictions 

# Display sample data
transformed_market_df_copy.head()

In [17]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

scaled_plot=transformed_market_df_copy.hvplot.scatter(
                    x="price_change_percentage_24h",
                    y="price_change_percentage_7d",
                    by="Prediction",
                    hover_cols=['Crypto_names']
                )

scaled_plot

---

### Optimize Clusters with Principal Component Analysis.

In [18]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [19]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_model = pca.fit_transform(transformed_market_df)
# View the first five rows of the DataFrame. 
pca_model[:5]

In [20]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

In [21]:
#Find % 
import numpy as np
total_explained_variance = np.sum(pca.explained_variance_ratio_)
total_explained_variance

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89%

In [22]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(
    pca_model,
    columns = ["PCA1", "PCA2" , "PCA3"]
)

# Copy the crypto names from the original data
pca_df["Crypto_names"] = df_market_data.index

# Set the coinid column as index
pca_df = pca_df.set_index('Crypto_names')

# Display sample data
pca_df.head()

---

### Find the Best Value for k Using the PCA Data

In [23]:
# Create a list with the number of k-values from 1 to 11
k_pca=list(range(1,11))

In [24]:
# Create an empty list to store the inertia values
inertia=[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k_pca:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(pca_df)
    inertia.append(k_model.inertia_)

In [25]:
# Create a dictionary with the data to plot the Elbow curve
elbow_dict = {'k_pca': k_pca , 'inertia': inertia}

# Create a DataFrame with the data to plot the Elbow curve
pca_elbow_df = pd.DataFrame(elbow_dict)
pca_elbow_df.head()

In [26]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
pca_elbow_plot = pca_elbow_df.hvplot.line(
    x="k_pca", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)
pca_elbow_plot

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Both sets of data have the value of 4 for K

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [27]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=2, random_state=0)

In [28]:
# Fit the K-Means model using the PCA data
model.fit(pca_df)

In [29]:
# Predict the clusters to group the cryptocurrencies using the PCA data
k_predict = model.predict(pca_df)

# Print the resulting array of cluster values.
k_predict

In [30]:
# Create a copy of the DataFrame with the PCA data
pca_prediction_df = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_prediction_df['pca_prediction']= k_predict

# Display sample data
pca_prediction_df.head()

In [31]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_plot = pca_prediction_df.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by="pca_prediction",
    hover_cols=['Crypto_names']
)
pca_plot

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [32]:
# Composite plot to contrast the Elbow curves
elbow_curve_plot + pca_elbow_plot 

In [33]:
# Composite plot to contrast the clusters
scaled_plot + pca_plot 

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** After visually analysing the cluster analysis results, the impact of using two clusters gives us a clear and more concise cluster of data. 